In [1]:
import pandas as pd
import json
import geopandas as gpd
import osmnx as ox
import folium

In [2]:
bridge_gdf = gpd.read_file('gis_data/River_Bridges.geojson')
bridge_gdf

,OBJECTID,NAME,PAVED,Shape_Area,Shape_Length,source,geometry
0,1,St. Johns,1,34223.144859,3240.751985,BPS,"POLYGON ((-122.75740 45.58867, -122.76229 45.5..."
1,2,Fremont,1,208696.294212,11166.941617,BPS,"POLYGON ((-122.67664 45.54522, -122.67657 45.5..."
2,3,Broadway,1,45897.938410,3650.669179,BPS,"POLYGON ((-122.67203 45.53320, -122.67231 45.5..."
3,4,Steel,1,37016.839907,3257.801335,BPS,"POLYGON ((-122.66723 45.52927, -122.66723 45.5..."
4,5,Burnside,1,27399.241362,1631.367688,BPS,"POLYGON ((-122.67024 45.52322, -122.67090 45.5..."
5,6,Morrison,1,46860.849077,4231.251089,BPS,"POLYGON ((-122.67175 45.51940, -122.67195 45.5..."
6,7,Hawthorne,1,38241.095246,3449.418898,BPS,"POLYGON ((-122.67498 45.51504, -122.67500 45.5..."
7,8,Marquam,1,70570.030703,4949.266392,BPS,"POLYGON ((-122.66700 45.51166, -122.66749 45.5..."
8,9,Ross Island,1,50630.206240,4844.145245,BPS,"POLYGON ((-122.66177 45.50152, -122.66220 45.5..."
9,10,Sellwood,1,10946.935688,2192.461442,BPS,"POLYGON ((-122.66921 45.46508, -122.66922 45.4..."


In [3]:
m = folium.Map(location=[45.5236, -122.6750], zoom_start=13)
folium.GeoJson(data=bridge_gdf['geometry']).add_to(m)
m

## Change Coordinate Reference System so we can buffer in a unit other than degrees
Going to use [UTM Zone 10](https://epsg.io/32610?msclkid=15e978fcd01a11ecb982387f5d9ea86b) - EPSG:32610 - unit meters

In [4]:
bridge_epsg32610 = bridge_gdf.to_crs("EPSG:32610")

In [5]:
bridge_epsg32610['geometry'] = bridge_epsg32610['geometry'].apply(lambda x: x.buffer(25))

In [6]:
bridge_buffers_for_plot = bridge_epsg32610.to_crs("EPSG:4326")

In [7]:
bridge_buffers_for_plot.head(3)

,OBJECTID,NAME,PAVED,Shape_Area,Shape_Length,source,geometry
0,1,St. Johns,1,34223.144859,3240.751985,BPS,"POLYGON ((-122.76822 45.58308, -122.76822 45.5..."
1,2,Fremont,1,208696.294212,11166.941617,BPS,"POLYGON ((-122.69178 45.53511, -122.69216 45.5..."
2,3,Broadway,1,45897.938410,3650.669179,BPS,"POLYGON ((-122.68179 45.52979, -122.68329 45.5..."


In [8]:
m = folium.Map(location=[45.5236, -122.6750], zoom_start=13)
folium.GeoJson(data=bridge_buffers_for_plot[['geometry','NAME']], tooltip=folium.features.GeoJsonTooltip(
        fields=['NAME'] 
    )).add_to(m)

m

In [9]:
bridge_buffers_for_plot.to_file('gis_data/portland_river_bridge_buffer.geojson', driver='GeoJSON')